In [ ]:
import numpy as np
from time_templates.datareader.get_data import fetch_MC_data_from_tree
from time_templates.utilities.plot import plot_hist, plot_profile_1d, add_identity
from time_templates.utilities.misc import histedges_equalN
from time_templates.utilities.fitting import plot_fit_curve, fit_curve
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
from time_templates.templates.universality.names import DICT_COMP_SIGNALKEY, eEM_HAD, eEM_MU, eEM_PURE, eMUON, DICT_COMP_LABELS, DICT_COMP_COLORS, DICT_COMP_MARKERS
from time_templates.templates.universality.S1000_model import modified_Gaisser_Hillas
from time_templates.utilities.stats import get_kde_mode

SAVE = True
BOOTSTRAPS = 0
if SAVE:
    plt.style.use('science')
    BOOTSTRAPS = 1000
gcm2 = "\\mathrm{g/cm^2}"
label_DX = f"$\\Delta X \, [{gcm2}]$"
label_VEM = "$\\rm Signal\, [\\rm VEM]$"

DICT_FRAC_LABELS = {comp: f"$S_{{{label}}}(1000) / \\widehat{{S}}_{{{label}}}(1000)$" for comp, label in DICT_COMP_LABELS.items()}

In [ ]:
np.linspace(0.6, 1.4, 20)

In [ ]:
dfs = []
for energy in ['18.5_19', '19_19.5', '19.5_20']:
    df = fetch_MC_data_from_tree(primary='proton', energy=energy, no_traces=True, dense=True, force=False)
    dfs.append(df)
    
df = pd.concat(dfs)

In [ ]:
df = df.query("MCr_round == 1000 & MCDX_1000 > 10 & MCCosTheta > 0.5")

In [ ]:
nE = 12
nct2 = 6
lgE_bins = np.linspace(18.5, 20, nE)
lgE_bins = np.arange(18.5, 20.01, 0.1)
nE = lgE_bins
DXmax_bins = np.arange(0, 1200, 50)
costheta2_bins = np.linspace(0.25, 1, nct2)
df["MCCosTheta_sq_bin"] = pd.cut(df["MCCosTheta"] ** 2, costheta2_bins)
df["MClgE_bin"] = pd.cut(df["MClgE"], lgE_bins)

#Take DX as as 1000m at psi=+-np.pi/2=+-90 deg
df["MCDX_bin"] = pd.cut(df["MCDX_1000"], DXmax_bins)
df_gb = df.groupby(['MClgE_bin'])
# df_mean = df_gb.mean()
# df_std = df_gb.std()
# n = df_gb.size()
# df_n = df_gb.count()

In [ ]:
DX0_COMP = {eMUON: -250, eEM_PURE: -500, eEM_MU: -250, eEM_HAD: -800}

def S1000_func(DX, Sref, DXmax, DX0, lam):
    return Sref * modified_Gaisser_Hillas(DX, DXmax, DX0, lam)

In [ ]:
# f, axes = plt.subplots(2, 2, figsize=(12, 10))
f, ax = plt.subplots(1, figsize=(8, 6))
# axes = axes.flatten()
FIT_DX0 = False # Just don't

dd = defaultdict(list)

for lgE_bin, df_ in df_gb:
#     print(len(df_.groupby('EventId').count()))
    df_gb_ = df_.groupby('MCDX_bin')
    df_mean = df_gb_.mean()
    df_std = df_gb_.std()
    n = df_gb_.count()
    x = df_mean['MCDX_1000'].values

#     for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    for comp, signal_key in DICT_COMP_SIGNALKEY.items():
        y = df_mean[signal_key].values
#         y = df_gb_[signal_key].apply(get_kde_mode)
#         y = np.mean(df_gb_[])
#         y = df_gb_[signal_key].median().values
#         y[~np.isfinite(df_mean[signal_key].values)] = np.nan
#         y = y.astype(float)

        yerr = np.sqrt(df_std[signal_key].values**2/n[signal_key].values)# + 0.001*y)

        mask = np.isfinite(x*y*yerr) & (yerr > 0)
        if FIT_DX0:
            fitfunc = lambda DX, Sref, DXmax, lam, DX0: S1000_func(DX, Sref, DXmax, DX0, lam)
            p0 = [10**(1 + lgE_bin.mid-19), 300, 200, DX0_COMP[comp]]
        else:
            fitfunc = lambda DX, Sref, DXmax, lam: S1000_func(DX, Sref, DXmax, DX0_COMP[comp], lam)
            p0 = [10**(1 + lgE_bin.mid-19), 300, 200]
            
        comp_label = DICT_COMP_LABELS[comp]
        if 19.5 in lgE_bin:
            ax, (fitp, fitp_err, chi2, ndof) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=fitfunc,
                                                              p0=p0, ax=ax, smoother_x=True, ebar_kws=dict(color=DICT_COMP_COLORS[comp], ls='', marker=DICT_COMP_MARKERS[comp]),
                                                              param_names=[f'S_{{{comp_label}}}^{{\\rm ref}}', '\Delta X_{\\rm max}', '\\lambda'],
                                                             units=['\\rm VEM', gcm2, gcm2], custom_label=f"${comp_label}$")
#             print(lgE_bin)
        else:
            fitp, fitp_err, chi2, ndof = fit_curve(x[mask], y[mask], yerr=yerr[mask], func=fitfunc,
                                                  p0=p0)
        if FIT_DX0:
            Sref, DXmax, lam, DX0 = fitp
            Sref_err, DXmax_err, lam_err, DX0_err = fitp_err
            dd['DX0'].append(DX0)
            dd['DX0_err'].append(DX0_err)
        else:
            Sref, DXmax, lam = fitp
            Sref_err, DXmax_err, lam_err = fitp_err

        dd['lgE'].append(lgE_bin.mid)
        dd['comp'].append(comp)
        dd['Sref'].append(Sref)
        dd['DXmax'].append(DXmax)
        dd['lam'].append(lam)
        dd['Sref_err'].append(Sref_err)
        dd['DXmax_err'].append(DXmax_err)
        dd['lam_err'].append(lam_err)
        dd['chi2'].append(chi2)
        dd['ndof'].append(ndof)
#         ax.set_title(f"${DICT_COMP_LABELS[comp]}$")
ax.set_xlabel(label_DX)
ax.set_ylabel(label_VEM)      
# ax.legend(fontsize=12, ncol=2)
labels = [f"${lab}$" for lab in DICT_COMP_LABELS.values()]
from matplotlib.lines import Line2D
handles = [Line2D([0], [0], color=DICT_COMP_COLORS[key], marker=DICT_COMP_MARKERS[key]) for key in DICT_COMP_LABELS.keys()]
ax.legend(handles, labels, loc=3)
ax.set_yscale('log')

if SAVE:
    plt.savefig('DXmax_vs_signal_example.pdf', bbox_inches='tight')    
        
df_fit = pd.DataFrame(dd)

In [ ]:
df_fit['lgSref'] = np.log10(df_fit['Sref'])
df_fit['lgSref_err'] = df_fit['Sref_err']/df_fit['Sref']*1/np.log(10)
# df_fit = df_fit.query('lgE >= 19')

dict_GH_params = {}
dict_param_keys = {'lgSref': lambda x: f"$\\log_{{10}}\\left(S_{x}^{{\\rm ref}} \\right)$", 
                  "DXmax": lambda x: "$\\Delta X_{\\rm max}$",
                   "lam": lambda x: "$\\lambda$"}
f, axes = plt.subplots(4, 3, figsize=(12, 12))

for irow, comp in enumerate(DICT_COMP_SIGNALKEY.keys()):
    df_ = df_fit.query(f'comp == "{comp}"')
    axes_row = axes[irow]
    x = df_['lgE']
    dict_GH_params[comp] = {}
    for ax, (key, label) in zip(axes_row, dict_param_keys.items()):
        y = df_[key]
        yerr = df_[key+"_err"]*np.sqrt(df_['chi2']/df_['ndof'])
        if key == 'lgSref' or (key == 'lam' and (comp == eEM_PURE or comp == eEM_HAD)):
            _, (fitp, fitp_err, chi2_1, ndof_1) = plot_fit_curve(x, y, yerr=yerr, func=lambda x, a, b: a+b*(x-19), ax=ax)
            a, b = fitp
        else:
            _, (fitp, fitp_err, chi2_0, ndof_0) = plot_fit_curve(x, y, yerr=yerr, func=lambda x, a: a*np.ones_like(x), ax=ax)
            a = fitp[0]
            b = 0

        dict_GH_params[comp][key] = [a, b]
        
        ax.set_ylabel(label(DICT_COMP_LABELS[comp]))
        ax.set_xlabel('$\\log_{10}\\left(E/ \\rm eV \\right)$')
        ax.legend(fontsize=10)
        
plt.subplots_adjust(wspace=0.4, hspace=0.4)
if SAVE:
    plt.savefig('S1000_params_lin_lgE.pdf', bbox_inches='tight')

In [ ]:
import json

with open("shower_size_GH_fit_params.json", "w") as outfile:
    json.dump(dict_GH_params, outfile)

dict_GH_params

In [ ]:
dfs = []
for energy in ['18.5_19', '19_19.5', '19.5_20']:
    df = fetch_MC_data_from_tree(primary='proton', energy=energy, no_traces=True, dense=True)
    dfs.append(df)
    df = fetch_MC_data_from_tree(primary='iron', energy=energy, no_traces=True, dense=True, force=False)
    dfs.append(df)
    
df = pd.concat(dfs)
df = df.query('MCr_round == 1000 & MCDXstation > 10 & MCCosTheta >= 0.5')
df_gb = df.groupby('EventId')
df = df_gb.mean()
df['primary'] = df_gb['primary'].min()

In [ ]:
def lin_lgE(lgE, a, b):
    return a + b * (lgE-19)

lgE = df['MClgE'].values
DXmax = df['MCDX_1000'].values
for key, val in dict_GH_params.items():
    if FIT_DX0:
        df[key+"_pred"] = S1000_func(DXmax, 10**lin_lgE(lgE, *val['lgSref']), lin_lgE(lgE, *val['DXmax']), lin_lgE(lgE, *val['DX0']), lin_lgE(lgE, *val['lam']))
    else:
        df[key+"_pred"] = S1000_func(DXmax, 10**lin_lgE(lgE, *val['lgSref']), lin_lgE(lgE, *val['DXmax']), DX0_COMP[key], lin_lgE(lgE, *val['lam']))
    df[key+"_ratio"] = df[DICT_COMP_SIGNALKEY[key]]/df[key+"_pred"]
    df[key+"_diff"] = 2*(df[DICT_COMP_SIGNALKEY[key]]-df[key+"_pred"])/(df[DICT_COMP_SIGNALKEY[key]]+df[key+"_pred"])
df_p = df.query("primary == 'proton'")
df_Fe = df.query("primary == 'iron'")

In [ ]:
def sec_theta_correction(sec_theta, a, b, c, d):
    x = sec_theta - 1.33
    return a + b*x + c*x**2 + d*x**3

dict_sec_theta_corr = {}

f, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

bins = histedges_equalN(df_p['MCSecTheta'], 15)

for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    ax, (x, y, yerr) = plot_profile_1d(df_p['MCSecTheta'], df_p[comp+"_ratio"], ax=ax, bins=bins, color=DICT_COMP_COLORS[comp], stat='mean', bootstraps=BOOTSTRAPS)
    ax, (fitp, _, _, _ ) = plot_fit_curve(x, y, yerr=yerr, func=sec_theta_correction, ax=ax, ebar_kws=dict(alpha=0, color=DICT_COMP_COLORS[comp]), smoother_x=True)
    dict_sec_theta_corr[comp] = list(fitp)
#     ax.set_title(DICT_COMP_LABELS[comp])
    ax.set_xlabel('$\\sec{\\theta}$')
    ax.set_ylabel(DICT_FRAC_LABELS[comp])
    ax.legend(fontsize=14)
    ax.set_ylim([0.8, 1.2])
    ax.axhline(1, ls='--', color='k')
    ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
    ax.set_xlim([1, 2])
    ax_top = ax.twiny()
    ax_top.set_xlim(ax.get_xlim())
    new_tick_locations = np.array([1, 1.15, 1.3, 1.55, 2])
    ax_top.set_xticks(new_tick_locations)
    new_tick_labels = np.rad2deg(np.arccos(1/new_tick_locations))
    ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
#     ax_top.set_xlabel('$\\theta$ [deg]')
    ax_top.tick_params(axis="x", which="minor", direction="in", 
                          top=False, labeltop=True, bottom=False, labelbottom=False)
    
plt.subplots_adjust(hspace=0.35, wspace=0.3)
if SAVE:
    plt.savefig('signal_ratio_sec_theta_dependence.pdf', bbox_inches='tight')

In [ ]:
with open("shower_size_sec_theta_corr.json", "w") as outfile:
    json.dump(dict_sec_theta_corr, outfile)

dict_sec_theta_corr

In [ ]:
from S1000_model import S1000_comp_model

lgE = df['MClgE'].values
DX = df['MCDX_1000'].values
theta = df['MCTheta'].values
df['total_pred'] = 0
for comp in DICT_COMP_LABELS:
    df[comp+"_pred"] = S1000_comp_model(lgE, DX, theta, 1, comp=comp)
    df[comp+"_ratio"] = df[DICT_COMP_SIGNALKEY[comp]]/df[comp+"_pred"]
    df['total_pred'] += df[comp+"_pred"]
df['total_ratio'] = df['WCDTotalSignal']/df['total_pred']
df['Rmu'] = df[eMUON+"_ratio"]

In [ ]:
# lgE = df['MClgE'].values
# DXmax = df['MCDXstation'].values
# sec_theta = df['MCSecTheta'].values
# for key, val in dict_GH_params.items():
#     df[key+"_pred"] = S1000_func(DXmax, 10**lin_lgE(lgE, *val['lgSref']), lin_lgE(lgE, *val['DXmax']), DX0_COMP[key], lin_lgE(lgE, *val['lam'])) * sec_theta_correction(sec_theta, *dict_sec_theta_corr[key])
#     df[key+"_ratio"] = df[DICT_COMP_SIGNALKEY[key]]/df[key+"_pred"]
#     df[key+"_diff"] = 2*(df[DICT_COMP_SIGNALKEY[key]]-df[key+"_pred"])/(df[DICT_COMP_SIGNALKEY[key]]+df[key+"_pred"])
# df['Rmu'] = df[eMUON+"_ratio"]

In [ ]:
df_p = df.query("primary == 'proton'")
df_Fe = df.query("primary == 'iron'")

In [ ]:
df_Fe['Rmu'].mean()

In [ ]:
def Rmu_func(Rmu, b):
    return (1 + b*(Rmu-1))

dict_Rmu_corr = {}

from time_templates.utilities.misc import histedges_equalN
# bins = histedges_equalN(df[eMUON+"_ratio"], 20)
bins = np.linspace(0.8, 1.5, 20)

f, axes = plt.subplots(1, 3, figsize=(15, 5))
axes = axes.flatten()

iax = 0
for comp in DICT_COMP_SIGNALKEY.keys():
    ax = axes[iax]
    if comp != eMUON:
        iax += 1

    ax.scatter(df_p['Rmu'], df_p[comp+"_ratio"], color='b', alpha=0.02, marker='.')
    ax.scatter(df_Fe['Rmu'], df_Fe[comp+"_ratio"], color='r', alpha=0.02, marker='.')
    ax, (x, y, yerr) = plot_profile_1d(df[eMUON+'_ratio'], df[comp+"_ratio"], ax=ax, bins=bins, marker='', alpha=0, stat='mean', bootstraps=BOOTSTRAPS)
    mask = np.isfinite(x*y*yerr) & (yerr > 0)
    ax, (pfit, pfit_err, chi2, ndf) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=Rmu_func, ax=ax, ebar_kws=dict(color='k', ls='', marker='', alpha=0))
    dict_Rmu_corr[comp] = pfit[0]
    
    ax, (x, y, yerr) = plot_profile_1d(df_p['Rmu'], df_p[comp+"_ratio"], ax=ax, bins=histedges_equalN(df_p[eMUON+"_ratio"], 10), color='b')
    ax, (x, y, yerr) = plot_profile_1d(df_Fe['Rmu'], df_Fe[comp+"_ratio"], ax=ax, bins=histedges_equalN(df_Fe[eMUON+"_ratio"], 10), color='r')
    
    if comp == eEM_PURE:
        ax.axhline(1, ls=':', color='k')
    else:
        add_identity(ax, color='k', ls=':')
    ax.set_xlabel('$R^\\mu$')
    ax.set_ylabel(DICT_FRAC_LABELS[comp])
    ax.annotate(xy=(0.1, 0.95), text=f"$\\alpha_{{{DICT_COMP_LABELS[comp]}}} = {pfit[0]:.3f} \\pm {pfit_err[0]*np.sqrt(chi2/ndf):.3f}$", fontsize=14, xycoords='axes fraction',
                ha='left', va='top')
#     ax.legend()
    ax.set_ylim([0.6, 1.7])
    ax.set_xlim([0.6, 1.7])
    if comp == eMUON:
        ax.clear()
        
plt.subplots_adjust(wspace=0.3)
if SAVE:
    plt.savefig('signal_ratio_Rmu_dependence.pdf', bbox_inches='tight')

In [ ]:
dict_Rmu_corr[eMUON] = 1

with open("shower_size_Rmu_corr.json", "w") as outfile:
    json.dump(dict_Rmu_corr, outfile)

dict_Rmu_corr

In [ ]:
from S1000_model import S1000_comp_model

lgE = df['MClgE'].values
DX = df['MCDX_1000'].values
theta = df['MCTheta'].values
df['Rmu'] = 1
df['Rmu'].loc[df['primary'] == 'proton'] = 1
df['Rmu'].loc[df['primary'] == 'iron'] = 1.326
df['total_pred'] = 0
Rmu = df['Rmu'].values
for comp in DICT_COMP_LABELS:
    df[comp+"_pred"] = S1000_comp_model(lgE, DX, theta, Rmu, comp=comp)
    df[comp+"_ratio"] = df[DICT_COMP_SIGNALKEY[comp]]/df[comp+"_pred"]
    df['total_pred'] += df[comp+"_pred"]
df['total_ratio'] = df['WCDTotalSignal']/df['total_pred']

In [ ]:
df_p = df.query("primary == 'proton'")
df_Fe = df.query("primary == 'iron'")

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()
bins = np.linspace(0.5, 1.5, 50)
for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    plot_hist(df_p[comp+"_ratio"], ax=ax, bins=bins, histtype='step', color='b', density=True);
    plot_hist(df_Fe[comp+"_ratio"], ax=ax, bins=bins, histtype='step', color='r', density=True);
#     ax.set_title(DICT_COMP_LABELS[comp])
    ax.legend(fontsize=14, ncol=2)
    ax.axvline(1, ls='--', color='k')
    ax.set_ylabel(DICT_FRAC_LABELS[comp])
    ax.set_ylabel('probability density')
    
plt.subplots_adjust(hspace=0.3)
if SAVE:
    plt.savefig('signal_ratio_hist_Rmu_theta_corr.pdf', bbox_inches='tight')

In [ ]:
bins = np.arange(0.6, 1.4025, 0.025)
df_p_ = df_p.query('MCCosTheta > 0.5 & MClgE > 19.4 & MClgE < 19.6')
df_Fe_ = df_Fe.query('MCCosTheta > 0.5 & MClgE > 19.4 & MClgE < 19.6')

f, ax = plt.subplots(1)#, figsize=(8, 6))
plot_hist(df_p_['total_ratio'], color='b', bins=bins, histtype='step', fit_norm=False, ax=ax)
plot_hist(df_Fe_['total_ratio'], color='r', bins=bins, histtype='step', fit_norm=False, ax=ax)
ax.axvline(1, ls='--', color='k')

ax.legend(fontsize=14, ncol=1)
ax.axvline(1, ls='--', color='k')
ax.set_xlabel(f'$S_{{\\rm total}}(1000) / \\widehat{{S}}_{{\\rm total}}(1000)$')
ax.set_ylabel('probability density')
# ax.set_yscale('log')
if SAVE:
    plt.savefig('total_signal_ratio_hist.pdf')

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()
lgE_bins = np.arange(18.5, 20.01, 0.1)
for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    ax, (x, y, yerr) = plot_profile_1d(df_p['MClgE'], df_p[comp+"_ratio"], ax=ax, bins=lgE_bins, color='b', bootstraps=BOOTSTRAPS)
    ax, (x, y, yerr) = plot_profile_1d(df_Fe['MClgE'], df_Fe[comp+"_ratio"], ax=ax, bins=lgE_bins, color='r', bootstraps=BOOTSTRAPS)

    ax.set_ylim([0.8, 1.2])
    ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
    ax.axhline(1, ls='--', color='k')
    ax.set_ylabel(DICT_FRAC_LABELS[comp])
    ax.set_xlabel('$\\log_{10}\\left(E/ \\rm eV \\right)$')
plt.subplots_adjust(hspace=0.3, wspace=0.25)
if SAVE:
    plt.savefig('signal_ratio_comps_profile_all_corrected_lgE.pdf', bbox_inches='tight')

In [ ]:
f, ax = plt.subplots(1)

plot_profile_1d(df_p['MClgE'], df_p['total_ratio'], bins=lgE_bins, ax=ax, color='b', bootstraps=BOOTSTRAPS)
plot_profile_1d(df_Fe['MClgE'], df_Fe['total_ratio'], bins=lgE_bins, ax=ax, color='r', bootstraps=BOOTSTRAPS);
ax.set_ylim([0.8, 1.2])
ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
ax.axhline(1, ls='--', color='k')
ax.set_ylabel(f'$S_{{\\rm total}}(1000) / \\widehat{{S}}_{{\\rm total}}(1000)$')
ax.set_xlabel('$\\log_{10}\\left(E/ \\rm eV \\right)$')
if SAVE:
    plt.savefig('total_signal_ratio_profile_all_corrected_lgE.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

bins = histedges_equalN(df['MCSecTheta'], 15)
for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    ax, (x, y, yerr) = plot_profile_1d(df_p['MCSecTheta'], df_p[comp+"_ratio"], ax=ax, bins=bins, color='b', bootstraps=BOOTSTRAPS)
    ax, (x, y, yerr) = plot_profile_1d(df_Fe['MCSecTheta'], df_Fe[comp+"_ratio"], ax=ax, bins=bins, color='r', bootstraps=BOOTSTRAPS)

    ax.set_ylim([0.8, 1.2])
    ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
    ax.axhline(1, ls='--', color='k')
    ax.set_ylabel(DICT_FRAC_LABELS[comp])
    ax.set_xlabel('$\\sec{\\theta}$')
    
    ax_top = ax.twiny()
    ax_top.set_xlim(ax.get_xlim())
    new_tick_locations = np.array([1, 1.15, 1.3, 1.55, 2])
    ax_top.set_xticks(new_tick_locations)
    new_tick_labels = np.rad2deg(np.arccos(1/new_tick_locations))
    ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
    ax_top.tick_params(axis="x", which="minor", direction="in", 
                          top=False, labeltop=True, bottom=False, labelbottom=False)
plt.subplots_adjust(hspace=0.4, wspace=0.25)
if SAVE:
    plt.savefig('signal_ratio_comp_profile_all_corrected_sec_theta.pdf', bbox_inches='tight')

In [ ]:
f, ax = plt.subplots(1)

plot_profile_1d(df_p['MCSecTheta'], df_p['total_ratio'], bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS)
plot_profile_1d(df_Fe['MCSecTheta'], df_Fe['total_ratio'], bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS)
ax.set_ylim([0.8, 1.2])
ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
ax.axhline(1, ls='--', color='k')
ax.set_ylabel(f'$S_{{\\rm total}}(1000) / \\widehat{{S}}_{{\\rm total}}(1000)$')
ax.set_xlabel('$\\sec{\\theta}$')

ax_top = ax.twiny()
ax_top.set_xlim(ax.get_xlim())
new_tick_locations = np.array([1, 1.15, 1.3, 1.55, 2])
ax_top.set_xticks(new_tick_locations)
new_tick_labels = np.rad2deg(np.arccos(1/new_tick_locations))
ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
ax_top.tick_params(axis="x", which="minor", direction="in", 
                      top=False, labeltop=True, bottom=False, labelbottom=False)
ax.axvspan(1.667, 2, color='k', hatch='//', alpha=0.4)
ax.set_xlim([1, 2])
if SAVE:
    plt.savefig('total_signal_ratio_profile_all_corrected_sec_theta.pdf')

In [ ]:
from time_templates.datareader.get_data import fetch_MC_data_from_tree

In [ ]:
fetch_MC_data_from_tree()